In [1]:
!pip install ipython-sql
%load_ext sql
%sql sqlite://

In [2]:
%%sql
CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');
 

CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');
  

CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

 * sqlite://
Done.
15 rows affected.
Done.
3 rows affected.
Done.
2 rows affected.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [32]:
%%sql
-- Joining all things
select sales.customer_id,sales.order_date,menu.product_name,menu.price,
case when sales.order_date >= members.join_date then 'Y' else 'N' end as member
from 
sales
inner join
menu  on sales.product_id = menu.product_id
left join
members on sales.customer_id = members.customer_id;


 * sqlite://
Done.


customer_id,order_date,product_name,price,member
A,2021-01-01,sushi,10,N
A,2021-01-01,curry,15,N
A,2021-01-07,curry,15,Y
A,2021-01-10,ramen,12,Y
A,2021-01-11,ramen,12,Y
A,2021-01-11,ramen,12,Y
B,2021-01-01,curry,15,N
B,2021-01-02,curry,15,N
B,2021-01-04,sushi,10,N
B,2021-01-11,sushi,10,Y


In [39]:
%%sql
-- Joining all things
select t.*,case when t.member = 'Y' then rank() over(partition by t.customer_id,member order by t.order_date) else NULL end as ranking from
(select sales.customer_id,sales.order_date,menu.product_name,menu.price,
case when sales.order_date >= members.join_date then 'Y' else 'N' end as member
from 
sales
inner join
menu  on sales.product_id = menu.product_id
left join
members on sales.customer_id = members.customer_id) t;


 * sqlite://
Done.


customer_id,order_date,product_name,price,member,ranking
A,2021-01-01,sushi,10,N,None
A,2021-01-01,curry,15,N,None
A,2021-01-07,curry,15,Y,1
A,2021-01-10,ramen,12,Y,2
A,2021-01-11,ramen,12,Y,3
A,2021-01-11,ramen,12,Y,3
B,2021-01-01,curry,15,N,None
B,2021-01-02,curry,15,N,None
B,2021-01-04,sushi,10,N,None
B,2021-01-11,sushi,10,Y,1
